In [ ]:
!pip -q install pyomo
from pyomo.environ import *
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.4 MB/s eta 0:00:00


In [ ]:

courses = {'C0':3,'C01':2,'C2':6,'C3':5,'C4':5,'C5':4,'C6':6,'C7':8,'C8':8,'C9':5,'C10':4}
credits = list(courses.values())

1.2. Using the Pyomo framework and cbc solver in Python, solve your MILP formulation and report optimal
registration.

In [ ]:
model = ConcreteModel()
model.x = Var(np.arange(11),domain = Binary)
model.constraints = ConstraintList()
c= np.array([3,2,6,5,5,4,6,8,8,5,4])

model.obj = Objective(expr = sum(c[i]*model.x[i] for i in range(11)),sense = maximize)

model.constraints.add(expr = sum(model.x[i] for i in range(11) ) <= 9 )
model.constraints.add(expr = sum(model.x[i] for i in range(11)) >= 5 )
model.constraints.add(expr = sum(c[i]*model.x[i] for i in range(11)) >= 30)
model.constraints.add(expr = model.x[2] + model.x[7] <= 1)
model.constraints.add(expr = model.x[4] + model.x[9] <= 1)
model.constraints.add(expr = model.x[2] + model.x[8] <= 1)
model.constraints.add(expr = model.x[0] - model.x[10] >= 0)
model.constraints.add(expr = model.x[0] + model.x[1] +model.x[2] >= 1)
model.constraints.add(expr = model.x[0] + model.x[1] +model.x[2] <= 2)
model.constraints.add(expr = model.x[5] + model.x[6] <= 1)
model.constraints.add(expr = model.x[3] + model.x[6] +model.x[9] <= 2)

In [ ]:
# installing cbc solver in collab session
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')
result = optCBC.solve(model)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

In [ ]:

# snip to code for a function to print optimal solution as an output
def OptResults(Model):
  # display solution
  print('\nObjective = ', Model.Objective())

  print('\nDecision Variables')
  for i in np.arange(11):
    print('x[',i,'] = ', Model.x[i].value)
  print('\nConstraints')
  Model.constraints.display()
OptResults(model)


Objective =  <pyomo.core.base.block._generic_component_decorator object at 0x78dd61c02dd0>

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0

Constraints
constraints : Size=11
    Key : Lower : Body : Upper
      1 :  None :  8.0 :   9.0
      2 :   5.0 :  8.0 :  None
      3 :  30.0 : 41.0 :  None
      4 :  None :  1.0 :   1.0
      5 :  None :  1.0 :   1.0
      6 :  None :  1.0 :   1.0
      7 :   0.0 :  0.0 :  None
      8 :   1.0 :  2.0 :  None
      9 :  None :  2.0 :   2.0
     10 :  None :  1.0 :   1.0
     11 :  None :  2.0 :   2.0


In [ ]:
model.obj()

41.0

1.3. Relax the above-constructed formulation to a linear program and solve for optimality. Report the
solution for the relaxed program and compare it with the solution MILP formulation. Comment on
about “can the solution of MILP be obtained by merely rounding the LP solution?”. Explain.

In [ ]:
modelr = ConcreteModel()
modelr.x = Var(np.arange(11),domain = (0,1))
modelr.constraints = ConstraintList()
c= np.array([3,2,6,5,5,4,6,8,8,5,4])

modelr.obj = Objective(expr = sum(c[i]*modelr.x[i] for i in range(11)),sense = maximize)

modelr.constraints.add(expr = sum(modelr.x[i] for i in range(11) ) <= 9 )
modelr.constraints.add(expr = sum(modelr.x[i] for i in range(11)) >= 5 )
modelr.constraints.add(expr = sum(c[i]*modelr.x[i] for i in range(11)) >= 30)
modelr.constraints.add(expr = modelr.x[2] + modelr.x[7] <= 1)
modelr.constraints.add(expr = modelr.x[4] + modelr.x[9] <= 1)
modelr.constraints.add(expr = modelr.x[2] + modelr.x[8] <= 1)
modelr.constraints.add(expr = modelr.x[0] - modelr.x[10] >= 0)
modelr.constraints.add(expr = modelr.x[0] + modelr.x[1] +modelr.x[2] >= 1)
modelr.constraints.add(expr = modelr.x[0] + modelr.x[1] +modelr.x[2] <= 2)
modelr.constraints.add(expr = modelr.x[5] + modelr.x[6] <= 1)
modelr.constraints.add(expr = modelr.x[3] + modelr.x[6] +modelr.x[9] <= 2)

In [ ]:
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')
result = optCBC.solve(modelr)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
def OptResults(Model):
  # display solution
  print('\nObjective = ', Model.Objective())

  print('\nDecision Variables')
  for i in np.arange(11):
    print('x[',i,'] = ', Model.x[i].value)
  print('\nConstraints')
  Model.constraints.display()
OptResults(modelr)


Objective =  <pyomo.core.base.block._generic_component_decorator object at 0x78dd4e565570>

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0

Constraints
constraints : Size=11
    Key : Lower : Body : Upper
      1 :  None :  8.0 :   9.0
      2 :   5.0 :  8.0 :  None
      3 :  30.0 : 41.0 :  None
      4 :  None :  1.0 :   1.0
      5 :  None :  1.0 :   1.0
      6 :  None :  1.0 :   1.0
      7 :   0.0 :  0.0 :  None
      8 :   1.0 :  2.0 :  None
      9 :  None :  2.0 :   2.0
     10 :  None :  1.0 :   1.0
     11 :  None :  2.0 :   2.0


In [ ]:
modelr.obj()

41.0

1.4. A faculty in IEOR lately proposed an advanced course as C11 with 9 credits. But this course clashes
slot with C7, C8 and C9. Modify the above MILP formulation by adding this additional constraint.
Solve your modified MILP for the optimal solution.

In [ ]:
modelnew = ConcreteModel()
modelnew.x = Var(np.arange(12),domain = Binary)
modelnew.constraints = ConstraintList()
c= np.array([3,2,6,5,5,4,6,8,8,5,4,9])

modelnew.obj = Objective(expr = sum(c[i]*modelnew.x[i] for i in range(11)),sense = maximize)

modelnew.constraints.add(expr = sum(modelnew.x[i] for i in range(11) ) <= 9 )
modelnew.constraints.add(expr = sum(modelnew.x[i] for i in range(11)) >= 5 )
modelnew.constraints.add(expr = sum(c[i]*modelnew.x[i] for i in range(11)) >= 30)
modelnew.constraints.add(expr = modelnew.x[2] + modelnew.x[7] <= 1)
modelnew.constraints.add(expr = modelnew.x[4] + modelnew.x[9] <= 1)
modelnew.constraints.add(expr = modelnew.x[2] + modelnew.x[8] <= 1)
modelnew.constraints.add(expr = modelnew.x[0] - modelnew.x[10] >= 0)
modelnew.constraints.add(expr = modelnew.x[0] + modelnew.x[1] +modelnew.x[2] >= 1)
modelnew.constraints.add(expr = modelnew.x[0] + modelnew.x[1] +modelnew.x[2] <= 2)
modelnew.constraints.add(expr = modelnew.x[5] + modelnew.x[6] <= 1)
modelnew.constraints.add(expr = modelnew.x[3] + modelnew.x[6] +modelnew.x[9] <= 2)
modelnew.constraints.add(expr = modelnew.x[3] + modelnew.x[6] +modelnew.x[9] <= 2)
modelnew.constraints.add(expr = modelnew.x[11] + modelnew.x[8] <= 1)
modelnew.constraints.add(expr = modelnew.x[11] + modelnew.x[9] <= 1)
modelnew.constraints.add(expr = modelnew.x[11] + modelnew.x[7] <= 1)

In [ ]:
resultnew = optCBC.solve(modelnew)

print('Solver status:', resultnew.solver.status)
print('Solver termination condition:',resultnew.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
def OptResults(Modelnew):
  # display solution
  print('\nObjective = ', Modelnew.Objective())

  print('\nDecision Variables')
  for i in np.arange(12):
    print('x[',i,'] = ', Modelnew.x[i].value)
  print('\nConstraints')
  Modelnew.constraints.display()
OptResults(modelnew)


Objective =  <pyomo.core.base.block._generic_component_decorator object at 0x78dd4e84b3a0>

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0
x[ 11 ] =  0.0

Constraints
constraints : Size=15
    Key : Lower : Body : Upper
      1 :  None :  8.0 :   9.0
      2 :   5.0 :  8.0 :  None
      3 :  30.0 : 41.0 :  None
      4 :  None :  1.0 :   1.0
      5 :  None :  1.0 :   1.0
      6 :  None :  1.0 :   1.0
      7 :   0.0 :  0.0 :  None
      8 :   1.0 :  2.0 :  None
      9 :  None :  2.0 :   2.0
     10 :  None :  1.0 :   1.0
     11 :  None :  2.0 :   2.0
     12 :  None :  2.0 :   2.0
     13 :  None :  1.0 :   1.0
     14 :  None :  0.0 :   1.0
     15 :  None :  1.0 :   1.0


In [ ]:
modelnew.obj()

41.0

5. Relax this modified MILP to a simple linear program and check if the modified MILP can be obtained
by merely rounding the solution of the corresponding modified linear program

In [ ]:
modelnewr = ConcreteModel()
modelnewr.x = Var(np.arange(12),domain = (0,1))
modelnewr.constraints = ConstraintList()
c= np.array([3,2,6,5,5,4,6,8,8,5,4,9])

modelnewr.obj = Objective(expr = sum(c[i]*modelnewr.x[i] for i in range(11)),sense = maximize)

modelnewr.constraints.add(expr = sum(modelnewr.x[i] for i in range(11) ) <= 9 )
modelnewr.constraints.add(expr = sum(modelnewr.x[i] for i in range(11)) >= 5 )
modelnewr.constraints.add(expr = sum(c[i]*modelnewr.x[i] for i in range(11)) >= 30)
modelnewr.constraints.add(expr = modelnewr.x[2] + modelnewr.x[7] <= 1)
modelnewr.constraints.add(expr = modelnewr.x[4] + modelnewr.x[9] <= 1)
modelnewr.constraints.add(expr = modelnewr.x[2] + modelnewr.x[8] <= 1)
modelnewr.constraints.add(expr = modelnewr.x[0] - modelnewr.x[10] >= 0)
modelnewr.constraints.add(expr = modelnewr.x[0] + modelnewr.x[1] +modelnewr.x[2] >= 1)
modelnewr.constraints.add(expr = modelnewr.x[0] + modelnewr.x[1] +modelnewr.x[2] <= 2)
modelnewr.constraints.add(expr = modelnewr.x[5] + modelnewr.x[6] <= 1)
modelnewr.constraints.add(expr = modelnewr.x[3] + modelnewr.x[6] +modelnewr.x[9] <= 2)
modelnewr.constraints.add(expr = modelnewr.x[3] + modelnewr.x[6] +modelnewr.x[9] <= 2)
modelnewr.constraints.add(expr = modelnewr.x[11] + modelnewr.x[8] <= 1)
modelnewr.constraints.add(expr = modelnewr.x[11] + modelnewr.x[9] <= 1)
modelnewr.constraints.add(expr = modelnewr.x[11] + modelnewr.x[7] <= 1)

In [ ]:
resultnewr = optCBC.solve(modelnew)

print('Solver status:', resultnewr.solver.status)
print('Solver termination condition:',resultnewr.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
OptResults(modelnewr)


Objective =  <pyomo.core.base.block._generic_component_decorator object at 0x78dd4e5e9180>

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0
x[ 11 ] =  0.0

Constraints
constraints : Size=15
    Key : Lower : Body : Upper
      1 :  None :  8.0 :   9.0
      2 :   5.0 :  8.0 :  None
      3 :  30.0 : 41.0 :  None
      4 :  None :  1.0 :   1.0
      5 :  None :  1.0 :   1.0
      6 :  None :  1.0 :   1.0
      7 :   0.0 :  0.0 :  None
      8 :   1.0 :  2.0 :  None
      9 :  None :  2.0 :   2.0
     10 :  None :  1.0 :   1.0
     11 :  None :  2.0 :   2.0
     12 :  None :  2.0 :   2.0
     13 :  None :  1.0 :   1.0
     14 :  None :  0.0 :   1.0
     15 :  None :  1.0 :   1.0


In [ ]:
modelnewr.obj()

41.0

2.2. Create a model using the Pyomo framework to solve your proposed formulation. In your code, remember to specify which variables are integers. Use cbc solver to solve and report the optimal solution.

In [ ]:
model1 = ConcreteModel()
model1.x = Var(np.arange(5),domain = NonNegativeIntegers)
model1.constraints = ConstraintList()

model1.obj = Objective(expr = 12*model1.x[0] + 14*model1.x[1] + 10*model1.x[2] + 15*model1.x[3] + 14*model1.x[4],sense = maximize)

model1.constraints.add(expr = 6*model1.x[0] + 7.5*model1.x[1] + 9.5*model1.x[2] + 10*model1.x[3] + 12.5*model1.x[4] <= 59 )
model1.constraints.add(expr = 4.25*model1.x[0] + 3.25*model1.x[1] + 2.5*model1.x[2] + 3.4*model1.x[3] + 2.25*model1.x[4] <= 73)

result1 = optCBC.solve(model1)
print('Solver status:', result1.solver.status)
print('Solver termination condition:',result1.solver.termination_condition)


Solver status: ok
Solver termination condition: optimal


In [ ]:
def OptResults1(Model):
  # display solution
  print('\nObjective = ', Model.Objective())

  print('\nDecision Variables')
  for i in np.arange(5):
    print('x[',i,'] = ', Model.x[i].value)
  print('\nConstraints')
  Model.constraints.display()
OptResults1(model1)


Objective =  <pyomo.core.base.block._generic_component_decorator object at 0x78dd61be1870>

Decision Variables
x[ 0 ] =  6.0
x[ 1 ] =  3.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body  : Upper
      1 :  None :  58.5 :  59.0
      2 :  None : 35.25 :  73.0


In [ ]:
model1.obj()

114.0

2.3. Let us compare the above results to a linear program. Suppose you relax variables in the above problem
from being integers. Solve this relaxed problem to report optimal value.

In [ ]:
model1r = ConcreteModel()
model1r.x = Var(np.arange(5),domain = NonNegativeReals)
model1r.constraints = ConstraintList()

model1r.obj = Objective(expr = 12*model1r.x[0] + 14*model1r.x[1] + 10*model1r.x[2] + 15*model1r.x[3] + 14*model1r.x[4],sense = maximize)

model1r.constraints.add(expr = 6*model1r.x[0] + 7.5*model1r.x[1] + 9.5*model1r.x[2] + 10*model1r.x[3] + 12.5*model1r.x[4] <= 59 )
model1r.constraints.add(expr = 4.25*model1r.x[0] + 3.25*model1r.x[1] + 2.5*model1r.x[2] + 3.4*model1r.x[3] + 2.25*model1r.x[4] <= 73)

result1r = optCBC.solve(model1r)
print('Solver status:', result1r.solver.status)
print('Solver termination condition:',result1.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
OptResults1(model1r)


Objective =  <pyomo.core.base.block._generic_component_decorator object at 0x78dd4e61a680>

Decision Variables
x[ 0 ] =  9.8333333
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body         : Upper
      1 :  None :   58.9999998 :  59.0
      2 :  None : 41.791666525 :  73.0


In [ ]:
model1r.obj()

117.9999996

2.5. Suppose now we are interested in finding how the solution changes when the right-hand sides in our
constraints change. The government approves to increase budget allocation for new order to 71 crores
of rupees, and parking space is increased to 87 units. How additional resource constraints change
optimal values in both LP and MILP?


In [ ]:
model1new = ConcreteModel()
model1new.x = Var(np.arange(5),domain = NonNegativeIntegers)
model1new.constraints = ConstraintList()

model1new.obj = Objective(expr = 12*model1new.x[0] + 14*model1new.x[1] + 10*model1new.x[2] + 15*model1new.x[3] + 14*model1new.x[4],sense = maximize)

model1new.constraints.add(expr = 6*model1new.x[0] + 7.5*model1new.x[1] + 9.5*model1new.x[2] + 10*model1new.x[3] + 12.5*model1new.x[4] <= 71 )
model1new.constraints.add(expr = 4.25*model1new.x[0] + 3.25*model1new.x[1] + 2.5*model1new.x[2] + 3.4*model1new.x[3] + 2.25*model1new.x[4] <= 87)

result1new = optCBC.solve(model1new)
print('Solver status:', result1new.solver.status)
print('Solver termination condition:',result1.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
OptResults1(model1new)
print('Optimal solution',model1new.obj())


Objective =  <pyomo.core.base.block._generic_component_decorator object at 0x78dd61be3d90>

Decision Variables
x[ 0 ] =  8.0
x[ 1 ] =  3.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body  : Upper
      1 :  None :  70.5 :  71.0
      2 :  None : 43.75 :  87.0
Optimal solution 138.0


In [ ]:
# linear
model1new2 = ConcreteModel()
model1new2.x = Var(np.arange(5),domain = NonNegativeReals)
model1new2.constraints = ConstraintList()

model1new2.obj = Objective(expr = 12*model1new2.x[0] + 14*model1new2.x[1] + 10*model1new2.x[2] + 15*model1new2.x[3] + 14*model1new2.x[4],sense = maximize)

model1new2.constraints.add(expr = 6*model1new2.x[0] + 7.5*model1new2.x[1] + 9.5*model1new2.x[2] + 10*model1new2.x[3] + 12.5*model1new2.x[4] <= 71 )
model1new2.constraints.add(expr = 4.25*model1new2.x[0] + 3.25*model1new2.x[1] + 2.5*model1new2.x[2] + 3.4*model1new2.x[3] + 2.25*model1new2.x[4] <= 87)

result1new2 = optCBC.solve(model1new2)
print('Solver status:', result1new2.solver.status)
print('Solver termination condition:',result1new2.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
OptResults1(model1new2)
print('Optimal solution',model1new2.obj())


Objective =  <pyomo.core.base.block._generic_component_decorator object at 0x78dd61be0b20>

Decision Variables
x[ 0 ] =  11.833333
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body        : Upper
      1 :  None :   70.999998 :  71.0
      2 :  None : 50.29166525 :  87.0
Optimal solution 141.999996


2.6. Now try decreasing the limit on a budget from 71 units to 69, 67, 65, 63, and 61 units. Correspondingly,
increase the parking space to 88, 89, 90 and 91 units. Solve the LP and MILP for each of these four
possibilities and comment on the pattern seen in optimal objective function values of the LP and MILP.
Also, comment on your observations about the optimal solutions obtained.

In [ ]:
M, N = 8, 8

# since variable are two dimensional ie., x_{ij}, therefore both row and column indicies are defined below
row_indicies = np.arange(M)
col_indicies = np.arange(N)

# matrix of cost cofficient
cost = np.array([[627, 617, 1270, 2072, 1415, 2943, 1064, 4265,], [159, 1432, 351, 3372, 237, 3373, 1456, 5059], [609, 1943, 77, 3365, 298, 3021, 1484, 4803],[2143, 671, 3876, 537, 1902, 1941, 1682, 3348],[2742, 2089, 1931, 2199, 1226, 739, 691, 1128],[2053, 860 ,2995, 68, 2069, 2065 ,993, 3342],[656, 817, 1245, 2141, 1534, 3351, 1137, 3887],[195, 1862, 1331, 2921, 402, 2825, 563, 4441]])


hub_demand= np.array([175, 400, 550, 70, 650, 100, 65, 85])
plant_cap = np.array([185, 350, 405,280,105,75,425,270])
# defining an empty model using pyomo
TransModel = ConcreteModel()

# defining varibles used in model x_{ij} (in total 4x3 = 12 variables)
TransModel.x = Var(row_indicies, col_indicies, domain= NonNegativeReals)

# setting up the objective function
TransModel.Objective = Objective(expr = sum(cost[i][j]*TransModel.x[i,j] for i in row_indicies for j in col_indicies), sense = minimize)

In [ ]:
TransModel.constraints = ConstraintList()

# adding constraint for production capacity
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for j in col_indicies) <= plant_cap[i]) for i in row_indicies]
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for i in row_indicies) == hub_demand[j]) for j in col_indicies]

# printing out pyomo model
TransModel.pprint()

4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
       

In [ ]:
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')
result = optCBC.solve(TransModel)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
import pandas as pd

temp = pd.DataFrame({'M1': [TransModel.x[i, 0].value for i in row_indicies],
                     'M2': [TransModel.x[i, 1].value for i in row_indicies],
                     'M3': [TransModel.x[i, 2].value for i in row_indicies],
                     'M4': [TransModel.x[i, 3].value for i in row_indicies],
                     'M5': [TransModel.x[i, 4].value for i in row_indicies],
                     'M6': [TransModel.x[i, 5].value for i in row_indicies],
                     'M7': [TransModel.x[i, 6].value for i in row_indicies],
                     'M8': [TransModel.x[i, 7].value for i in row_indicies],
                     })

In [ ]:
temp

M1     M2     M3    M4     M5    M6    M7    M8
0    0.0  185.0    0.0   0.0    0.0   0.0   0.0   0.0
1    0.0    0.0    0.0   0.0  350.0   0.0   0.0   0.0
2    0.0    0.0  375.0   0.0   30.0   0.0   0.0   0.0
3    0.0  200.0    0.0   0.0    0.0  80.0   0.0   0.0
4    0.0    0.0    0.0   0.0    0.0  20.0   0.0  85.0
5    0.0    0.0    0.0  70.0    0.0   0.0   5.0   0.0
6  175.0   15.0  175.0   0.0    0.0   0.0  60.0   0.0
7    0.0    0.0    0.0   0.0  270.0   0.0   0.0   0.0

In [ ]:
TransModel.Objective()

1166465.0